In [1]:
from multiprocessing import Pool
import os
import copy
import numpy as np

In [2]:
numprocs = os.environ.get('NUMPROCS', 16)
# pool = Pool(numprocs)
numprocs

16

In [6]:
D = 3
np.eye(D)*np.random.normal(0,1,size=D)

array([[ 2.30743564,  0.        , -0.        ],
       [ 0.        ,  0.23680516, -0.        ],
       [ 0.        ,  0.        , -0.07429716]])

In [ ]:
def generateTransitionMatrix(dim):
    A = np.eye(dim) - np.random.beta(2,1,size=dim)*np.eye(dim)  #+ 40*np.eye(dim) + 40*waterfall
    return A

def generateCovMatrix(dim, M, scale, reg):
    noise_cov = scale*stats.invwishart(scale=100*np.eye(3), df=100).rvs()
    return noise_cov

In [8]:
def run_experiment_ar(iter_number, data_choice=0, **kwargs):
    
    # set the starting parameters for each model and run the models
    starting_params = {}
    starting_params['pi']    = np.random.dirichlet(alpha=np.ones(L), size=L)
    starting_params['R']     = kwargs['R'] if 'R' in kwargs else 1e-1*np.eye(D)
    starting_params['R0']    = kwargs['R0'] if 'R0' in kwargs else 1e-1*np.eye(D)
    starting_params['r0']    = kwargs['r0'] if 'r0' in kwargs else 1e2
    starting_params['C']     = kwargs['C'] if 'C' in kwargs else np.eye(D)
    starting_params['D']     = len(Y[0])
    starting_params['theta'] = [{'A': np.array(np.eye(D)), 'sigma': 1*np.eye(D)} for i in range(L)]
    starting_params['L']     = 10
    starting_params['Y']     = Y
    starting_params['T']     = len(Y)
    starting_params['alpha'] = kwargs['alpha'] if 'alpha' in kwargs else 1
    starting_params['beta']  = np.random.dirichlet(np.ones(params['L']))
    starting_params['kappa'] = kwargs['kappa'] if 'kappa' in kwargs else len(Y)/20
    starting_params['gamma'] = kwargs['gamma'] if 'gamma' in kwargs else 1
    
    chain_params = [{"A": generateTransitionMatrix(dim), "sigma": generateCovMatrix(3, np.zeros(shape=(dim, dim)),1 ,5)}, 
                    {"A": generateTransitionMatrix(dim), "sigma": generateCovMatrix(3, np.zeros(shape=(dim, dim)),2 ,5)}, 
                    {"A": generateTransitionMatrix(dim), "sigma": generateCovMatrix(3, np.zeros(shape=(dim, dim)),5 ,5)}]

    
    ####################################################################
    # generate the data
    ####################################################################
    if data_choice == 0:
        res = gen.generate_data_nD(dim=3,
                                   num_chains=len(chain_params),
                                   length=kwargs['chain_length'] if 'chain_length' in kwargs else 1000,
                                   switch_prob=0.02,
                                   chain_params=chain_params)
        chains, Y = res['chains'], res['Y']
        
    elif data_choice == 1:
        chain_params = [{"A": [0.4, 0.4], "sigma": 1}, {"A": [0.7, 0.1], "sigma": 2}, {"A": [0.1, 0.7], "sigma": 4}]
        res = gen.generate_data_1D(num_chains=len(chain_params),
                                   length=kwargs['chain_length'] if 'chain_length' in kwargs else 1001,
                                   switch_prob=0.02,
                                   chain_params=chain_params,
                                   ar=2)
        chains, Y = res['chains'][1:], res['Y'][1:,0]
        
    elif data_choice == 2:
        chain_params = [{"A": generateTransitionMatrix(dim), "sigma": generateCovMatrix(3, np.zeros(shape=(dim, dim)), 4, 1)}, 
                        {"A": generateTransitionMatrix(dim), "sigma": generateCovMatrix(3, np.zeros(shape=(dim, dim)), 2, 1)}, 
                        {"A": generateTransitionMatrix(dim), "sigma": generateCovMatrix(3, np.zeros(shape=(dim, dim)), 1, 1)}]

        R = generateCovMatrix(dim, 0, .5, 1)
        res = gen.generate_data_slsd(dim=3,
                                     R=R,
                                     num_chains=len(chain_params),
                                     length=kwargs['chain_length'] if 'chain_length' in kwargs else 1000,
                                     switch_prob=0.02,
                                     chain_params=chain_params)
        chains, Y = res['chains'], res['Y']

    ####################################################################
    # run the models
    ####################################################################
    if ('exec_model' not in kwargs) or (('exec_model') in kwargs and 0 in kwargs['exec_model']):
    
        # run the AR1 model
        starting_params['priors'] = {
            'M': np.zeros(shape=(D,D), dtype=np.float64),
            'K': 10*np.eye(D, dtype=np.float64)
        }

        _, res, assignments_ar1, hamming_ar1 = hdp.sticky_Multi_HDP_AR(Y, copy.deepcopy(starting_params),
                                                 priors=[1*np.eye(D), D],
                                                 num_iter=kwargs['num_iter'] if 'num_iter' in kwargs else 5000,
                                                 return_assignments=True,
                                                 verbose=True,
                                                 chains=chains)
        print()
        print("********************************************************")
        print("******************DONE WITH AR(1)***********************")
        print("********************************************************")
        print()
        
    if ('exec_model' not in kwargs) or (('exec_model') in kwargs and 1 in kwargs['exec_model']):
        
        starting_params['priors'] = {
            # need to adjust the priors slightly
            'M': np.zeros(shape=(D,D*2), dtype=np.float64),
            'K': 10*np.eye(D*2, dtype=np.float64)
        }

        _, res, assignments_ar2, hamming_ar2 = hdp.sticky_Multi_HDP_AR2(Y, copy.deepcopy(starting_params),
                                                 priors=[1*np.eye(D), D],
                                                 num_iter=kwargs['num_iter'] if 'num_iter' in kwargs else 5000,
                                                 return_assignments=True,
                                                 verbose=True,
                                                 chains=chains)
    
        print()
        print("********************************************************")
        print("******************DONE WITH AR(2)***********************")
        print("********************************************************")
        print()
        
    if ('exec_model' not in kwargs) or (('exec_model') in kwargs and 2 in kwargs['exec_model']):
        
        if data_choice == 1:
            starting_params['C'] = np.array([1,0], dtype=np.float64)
            starting_params['priors'] = {
                'M': np.zeros(shape=(D,D), dtype=np.float64),
                'K': 10*np.eye(D, dtype=np.float64),
            }
        else:    
            starting_params['priors'] = {
                'M': np.zeros(shape=(D,D)),
                'K': 10*np.eye(D)
            }

        _, z, assignments_slds, hamming_slds = hdp.SLDS_blocked_sampler(Y, copy.deepcopy(starting_params),
                                                  priors=[1*np.eye(D), D],
                                                  num_iter=kwargs['num_iter'] if 'num_iter' in kwargs else 5000,
                                                  verbose=True,
                                                  return_assignments=True,
                                                  chains=chains)
        print()
        print("********************************************************")
        print("******************DONE WITH SLDS************************")
        print("********************************************************")
        print()
    
    print()
    print("********************************************************")
    print("******************DONE ITER: {}, Data #: {}*************".format(iter_number, data_choice))
    print("********************************************************")
    print()
    
    return {
        'data_choice': data_choice,
        'iter_number': iter_number,
        'hamming_ar1': hamming_ar1,
        'hamming_ar2': hamming_ar2,
        'hamming_slds': hamming_slds
    }

In [15]:
','.join(map(str, np.arange(0,16)))

'0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15'

In [66]:
def test(iter_number, data_choice=0, **kwargs):
    return {
        'data_choice': data_choice,
        'iter_number': iter_number,
        'hamming_ar1': 1*np.ones(10),
        'hamming_ar2': 2*np.ones(10),
        'hamming_slds': 3*np.ones(10)
    }

In [69]:
def run_multi_process(outfile, numprocs=4, epochs=100, **kwargs):
    
    pool = Pool(numprocs)
    
    results = []
    
    for i in range(epochs):
        results.append(pool.apply_async(test, args=(i, 0,), kwds=kwargs))
        results.append(pool.apply_async(test, args=(i, 1,), kwds=kwargs))
        results.append(pool.apply_async(test, args=(i, 2,), kwds=kwargs))
        
    if not os.path.isfile(outfile):
        with open(outfile, 'w') as out_f:
            out_f.write('data,iteration_number,algorithm\n')
            
    with open('/Users/nickhoernle/Desktop/test.csv', 'a') as out_f:
        for result in results:
            res = result.get()
            out_f.write('{data},{iter_no},"AR1",'.format(data=res['data_choice'], iter_no=res['iter_number']) + ','.join(map(str, res['hamming_ar1'])) + '\n')
            out_f.write('{data},{iter_no},"AR2",'.format(data=res['data_choice'], iter_no=res['iter_number']) + ','.join(map(str, res['hamming_ar2'])) + '\n')
            out_f.write('{data},{iter_no},"SLDS",'.format(data=res['data_choice'], iter_no=res['iter_number']) + ','.join(map(str, res['hamming_slds'])) + '\n')


In [70]:
kwargs = {
    'chain_length': 50,
    'num_iter': 20
}
run_multi_process('/Users/nickhoernle/Desktop/test.csv', **kwargs)

In [79]:
A_ = np.arange(0,18).reshape(3,-6).T.reshape(-1,)

In [82]:
A = np.array([a.T for a in A_.reshape(2, 3, -3)], dtype=np.float64)
A

array([[[ 0.,  1.,  2.],
        [ 6.,  7.,  8.],
        [12., 13., 14.]],

       [[ 3.,  4.,  5.],
        [ 9., 10., 11.],
        [15., 16., 17.]]])

In [7]:
np.exp(0)

1.0